In [164]:
import pandas as pd
from tqdm.auto import tqdm

## Emissions
This is the fixed emission code from part 2. By "fixed", we mean that it takes into consideration unknown words not in train set (i.e. replaced with `"#UNK#"`.

In [1]:
def train_emission(filename):
    """
    Returns - a dictionary containing emission parameters
    """
    with open(filename, encoding="utf8") as f:
        lines = f.readlines()
    
    # for each state y, keep track of each observation count i.e. count (y -> x)
    # before eg: {state1: {obs1: 1, obs2: 5}, state2: {obs1: 4}}
    emission_dict = {}
    
    # update emission_dict for state with count(y -> x) = 0
    # after eg: {state1: {obs1: 1, obs2: 5}, state2: {obs1: 4, obs2: 0}}
    observations = set()
    
    for line in lines:
        split_line = line.split()
        
        # process only valid lines
        if len(split_line) == 2:
            obs, state = split_line[0], split_line[1]
            
            observations.add(obs)
            
            if state not in emission_dict:
                emission_dict[state] = {}
                
            if obs not in emission_dict[state]:
                emission_dict[state][obs] = 1
            else:
                emission_dict[state][obs] += 1

    for k, v in emission_dict.items():
        for obs in observations:
            if obs not in v:
                emission_dict[k][obs] = 0
    
    return emission_dict

In [2]:
def get_emission_params_fixed(emission_dict, state, obs, k=0.5):
    
    if state not in emission_dict:
        raise Exception("State not in emission dict")
    
    state_data = emission_dict[state]
    count_y = sum(state_data.values()) # count(y)
    
    if obs == "#UNK#":
        count_y_to_x = k
    else:
        count_y_to_x = state_data[obs] # count(y -> x)
    
    return count_y_to_x / (count_y + k)

## Transitions 

In [3]:
def train_transition(filename):
    """
    Returns - a dictionary containing transition parameters
    """
    with open(filename, encoding="utf8") as f:
        lines = f.readlines()
    
    # for each state u, keep track of each state count i.e. count (u,v)
    # before eg: {START: {y1: 1, y2: 5}, y1: {y1: 3, y2: 4, STOP: 1}, y2: {y1: 1, STOP: 3}}
    transition_dict = {}
    
    # after eg: {START: {y1: 1, y2: 5, STOP: 0}, y1: {y1: 3, y2: 4, STOP: 1}, y2: {y1: 1, y2: 0, STOP: 3}}
    states = set()
    states.add('STOP')
    
    prev_state = 'START'
        
    for line in lines:
        split_line = line.split()
        
        if prev_state not in transition_dict:
            transition_dict[prev_state] = {}
                
        # can only be START or STOP
        if len(split_line) < 2:
            if 'STOP' not in transition_dict[prev_state]:
                transition_dict[prev_state]['STOP'] = 0
            
            transition_dict[prev_state]['STOP'] += 1
            prev_state = 'START'
        
        # processing the sentence
        elif len(split_line) == 2:
            curr_state = split_line[1]
            states.add(curr_state)
           
            if curr_state not in transition_dict[prev_state]:
                transition_dict[prev_state][curr_state] = 0
            
            transition_dict[prev_state][curr_state] += 1
            prev_state = curr_state
    
    for k, v in transition_dict.items():
        for state in states:
            if state not in v:
                transition_dict[k][state] = 0
    
    return transition_dict

In [4]:
def get_transition_params(transition_dict, u, v):
    
    if u not in transition_dict:
        raise Exception("State u not in transition dict")
        
    if v not in transition_dict[u]:
        raise Exception("State v not in transition dict")
    
    state_data = transition_dict[u]
    
    count_u_to_v = state_data[v] # count(u,v)
    count_u = sum(state_data.values()) # count(u)
            
    return count_u_to_v / count_u

### Testing with example

In [5]:
transition_dict = train_transition('../dataset/EN/train')

#transition_dict
get_transition_params(transition_dict, 'START', 'B-PP')

0.1087041628604985

In [11]:
def obtain_all_obs(emission_dict):
    """
    Obtain all distinct observations words in the emission_dict.
    Purpose: This helps us identify words in Test Set that do not exist in the Training Set (or the emission_dict)
    Returns - Set of Strings.
    """
    all_observations = set()
    
    for s_to_obs_dict in emission_dict.values():
        for obs in s_to_obs_dict.keys():
            all_observations.add(obs)
            
    return all_observations

def preprocess_sentence(sentence, training_set_words):
    """
    sentence - a list of Strings (words or observations)
    Returns - a list of Strings, where Strings not in training_set_words are replaced by "#UNK#"
    """
    return [ word if word in training_set_words else "#UNK#" for word in sentence ]

## Training

In [8]:
def train(filename):
    """
    Returns - A 2-tuple (Dict, Dict): emission and transition parameters
    """
    return train_emission(filename), train_transition(filename)

In [163]:
def viterbi(emission_dict, transition_dict, sentence, is_preprocessed):
    """
    Dynamic Programming approach (Viterbi algorithm) to generate state sequence given a sentence.
    emission_dict - Dictionary. Emission parameters generated from training data.
    transiiton_dict - Dictionary. Transition parameters generated from training data.
    sentence - list of Strings (input words or observations)
    is_preprocessed - boolean. True if variable sentence is preprocessed (unknown words (not in train) changed to "#UNK#")
    Returns - 3-tuple (DataFrame, DataFrame, List of Strings)
        Dataframe - Pi Table, |T| by n+2 dimension.
        Dataframe - Backtrace Table, |T| by n+2 dimension.
        List of Strings - (Predicted sequence of states corresponding to sentence).
    """
    
    all_states = list(emission_dict.keys())
    
    proc_sent = sentence
    if not is_preprocessed:
        training_set_words = obtain_all_obs(emission_dict)
        proc_sent = preprocess_sentence(sentence, training_set_words)
    proc_sent = ["start"] + proc_sent + ["stop"]
    
    n = len(proc_sent)

    # Pi Table
    P = pd.DataFrame(index=all_states + ["START", "STOP"], columns=range(n)).fillna(0)
    # Backtrace Table
    B = pd.DataFrame(index=all_states + ["START", "STOP"], columns=range(n))

    # Base Case
    P.loc['START', 0] = 1
    
    # Helper functions for recursive step
    a = lambda u, v: get_transition_params(transition_dict, u, v)
    b = lambda state, obs: get_emission_params_fixed(emission_dict, state, obs, k=0.5)

    # Recursive Forward Step
    for j in range(1, n-1): # Going right the columns (obs)
        x = proc_sent[j]  # Obtain j'th word in the (processed) sentence

        for v in all_states: # Going down the rows (states)
            for u in all_states + ["START"]:
                p = P.loc[u, j-1] * a(u, v) * b(v, x)
                if p > P.loc[v, j]:
                    P.loc[v, j] = p  # update probability
                    B.loc[v, j] = u  # update backpointer

    # Termination
    j = n - 1
    v = 'STOP'
    for u in all_states:
        p = P.loc[u, j-1] * a(u, v)
        if p > P.loc[v, j]:
            P.loc[v, j] = p  # probability
            B.loc[v, j] = u  # backpointer

    # Backtrace
    state_seq = ['STOP']
    for i in range(n-1, 0, -1):
        curr_state = state_seq[-1]
        prev_state = B.loc[curr_state, i]

        if pd.isnull(prev_state):  # No possible transition to STOP. Edge case.
            state_seq = ['O'] * n
            break

        state_seq.append(prev_state)
    state_seq = state_seq[::-1][1:-1]  # reverse and drop START, STOP
    
    return P, B, state_seq  

In [162]:
emission_dict, transition_dict = train(filename)

sentence = "He added that the stress-related compensation claims is about twice the average for all injury claims ."
sentence = sentence.split(' ')

_, _, seq = viterbi(emission_dict, transition_dict, sentence, is_preprocessed=False)
print(seq)

['B-NP', 'B-VP', 'B-SBAR', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'B-VP', 'B-ADJP', 'I-ADJP', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'O']


## Evaluaton on dev.in

In [167]:
sets = ['EN', 'SG', 'CN']
sets = [sets[0]]

for dataset in tqdm(sets):
    
    print(f"Evaluating on {dataset}.")
    
    in_file = f"../dataset/{dataset}/dev.in"
    train_file = f"../dataset/{dataset}/train"
    out_file = f"../dataset/{dataset}/dev.p3.out"
    
    # Train
    emission_dict, transition_dict = train(train_file)
    # Obtain all distinct words in Training Set
    training_set_words = obtain_all_obs(emission_dict)
    
    # Create file handler to write to /dev.p2.out
    outf_h = open(out_file, "w", encoding="utf8")
    
    # Read in file
    with open(in_file, encoding="utf8") as f:
        lines = f.readlines()
        
    sent = [] # initialise array to store 1 sentence at a time.
    for word in tqdm(lines):
        
        if word != "\n":
            sent.append(word.strip())
            
        # We reached end of sentence - time to predict sentence's sequence of states (aka tags)
        else:
            # preprocess sentence (change unknown words to "#UNK#")
            sent_proc = preprocess_sentence(sent, training_set_words)
            # obtain processed sentence's predicted state seq (list of corresponding predicted states for each word in sent)
            _, _, sent_state_sequence = viterbi(emission_dict, transition_dict, sent_proc, is_preprocessed=True)
            
            for word, state in zip(sent, sent_state_sequence):
                outf_h.write(word + ' ' + state)
                outf_h.write("\n") # newline for each word
            outf_h.write("\n") # another newline when end of sentence

            # Reset sentence list
            sent = []
            
    outf_h.close()  

Evaluating on EN.


KeyboardInterrupt: 

### Testing with example

In [108]:
filename = '../dataset/EN/train'
emission_dict, transition_dict = train(filename)

training_set_words = obtain_all_obs(emission_dict)
all_states = list(emission_dict.keys())

In [109]:
sentence = "The quick brown fox jumps over the lazy dog ."
sentence = sentence.split(' ')
proc_sent = preprocess_sentence(sentence, training_set_words)
print(sentence)
print(proc_sent)

proc_sent = ["start"] + proc_sent + ["stop"]
print(proc_sent)

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']
['The', 'quick', '#UNK#', 'fox', 'jumps', 'over', 'the', '#UNK#', 'dog', '.']
['start', 'The', 'quick', '#UNK#', 'fox', 'jumps', 'over', 'the', '#UNK#', 'dog', '.', 'stop']


In [151]:
n = len(proc_sent)

# Pi Table
P = pd.DataFrame(index=all_states + ["START", "STOP"], columns=range(n)).fillna(0)

# Backpropagate
B = pd.DataFrame(index=all_states + ["START", "STOP"], columns=range(n))

# Base Case
P.loc['START', 0] = 1

In [152]:
a = lambda u, v: get_transition_params(transition_dict, u, v)
b = lambda state, obs: get_emission_params_fixed(emission_dict, state, obs, k=0.5)

# Recursive Forward Step
for j in range(1, n-1): # Going right the columns (obs)
    x = proc_sent[j]  # Obtain j'th word in the (processed) sentence

    for v in all_states: # Going down the rows (states)
        for u in all_states + ["START"]:
            p = P.loc[u, j-1] * a(u, v) * b(v, x)
            if p > P.loc[v, j]:
                P.loc[v, j] = p  # update probability
                B.loc[v, j] = u  # update backpointer
    
# termination
j = n - 1
v = 'STOP'
for u in all_states:
    p = P.loc[u, j-1] * a(u, v)
    if p > P.loc[v, j]:
        P.loc[v, j] = p  # probability
        B.loc[v, j] = u  # backpointer
        
# backtrace
state_seq = ['STOP']
for i in range(n-1, 0, -1):
    curr_state = state_seq[-1]
    prev_state = B.loc[curr_state, i]
    
    if pd.isnull(prev_state):  # No possible transition to STOP. Edge case.
        state_seq = ['O'] * n
        break
        
    state_seq.append(prev_state)
state_seq = state_seq[::-1][1:-1]  # reverse and drop START, STOP

In [155]:
print(sentence)
print(state_seq)

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']
['B-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'O']
